In [1]:
!pip install opendatasets datasets accelerate rouge_score --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: wadiaatouhami
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs


100%|██████████| 400k/400k [00:00<00:00, 47.5MB/s]

In [2]:
import pandas as pd
from datasets import Dataset
import datasets
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import re

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
data = pd.read_csv("/content/nvidia-documentation-question-and-answer-pairs/NvidiaDocumentationQandApairs.csv")[["question", "answer"]]
data

,question,answer
0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...
1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...
2,What are some parallelization patterns mention...,The text mentions using parallelization patter...
3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...
4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...
...,...,...
7103,What is the focus of the GTC event in 2015?,The focus of the GTC event in 2015 is GPU code...
7104,How were the main changes made to the code for...,"The main changes included merging kernels, reg..."
7105,What are some key fields in the cudaDeviceProp...,"Some key fields include name, memoryClockRate,..."
7106,What did changing the kernel approach achieve ...,Changing the kernel approach reduced the itera...


In [4]:
def clean_text(text):
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text

data['question'] = data['question'].apply(clean_text)
data['answer'] = data['answer'].apply(clean_text)
data

,question,answer
0,what is hybridizer,hybridizer is a compiler from altimesh that en...
1,how does hybridizer generate optimized code,hybridizer uses decorated symbols to express p...
2,what are some parallelization patterns mention...,the text mentions using parallelization patter...
3,how can you benefit from accelerators without ...,you can benefit from accelerators compute hors...
4,what is an example of using hybridizer,an example in the text demonstrates using para...
...,...,...
7103,what is the focus of the gtc event in 2015,the focus of the gtc event in 2015 is gpu code...
7104,how were the main changes made to the code for...,the main changes included merging kernels regr...
7105,what are some key fields in the cudadeviceprop...,some key fields include name memoryclockrate a...
7106,what did changing the kernel approach achieve ...,changing the kernel approach reduced the itera...


In [5]:
train=data.sample(frac=0.7,random_state=7) # Create training of 70% of the data
test=data.drop(train.index) # Create testing by removing the 70% of the train data which will result in 30%

val=test.sample(frac=0.5,random_state=7) # Create validation of 50% of the testing data
test=test.drop(val.index) # Create testing by removing the 50% of the validation data which will result in 50%

In [6]:
print("Training Shape: ", train.shape)
print("Validation Shape: ", val.shape)
print("Testing Shape: ", test.shape)

Training Shape:  (4976, 2)
Validation Shape:  (1066, 2)
Testing Shape:  (1066, 2)


In [7]:
model_name = "google/flan-t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def tokenize_function(example):
    start_prompt = 'According to the following question:\n\n'
    end_prompt = '\n\nAnswer:\n\n'

    prompt = [start_prompt + question + end_prompt for question in example["question"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt", max_length=200).input_ids
    example['labels'] = tokenizer(example["answer"], padding="max_length", truncation=True, return_tensors="pt",max_length=200).input_ids

    return example

In [9]:
train_data = Dataset.from_pandas(train)
train_tokenized_datasets = train_data.map(tokenize_function, batched=True)
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

val_data = Dataset.from_pandas(val)
val_tokenized_datasets = val_data.map(tokenize_function, batched=True)
val_tokenized_datasets = val_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

test_data = Dataset.from_pandas(test)
test_tokenized_datasets = test_data.map(tokenize_function, batched=True)
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

Map:   0%|          | 0/4976 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [10]:
train_tokenized_datasets

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 4976
})

In [11]:
val_tokenized_datasets

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1066
})

In [12]:
test_tokenized_datasets

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1066
})

In [13]:
EPOCHS = 3
LR = 1e-3
BATCH_SIZE = 2

training_path = "./training_nvidia_chatbot"

training_args = TrainingArguments(
    output_dir = training_path,
    overwrite_output_dir = True,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LR,
    num_train_epochs = EPOCHS,
    evaluation_strategy = "epoch",
    save_total_limit = 2
    )

trainer= Trainer(
    model = original_model,
    args = training_args,
    train_dataset = train_tokenized_datasets,
    eval_dataset = val_tokenized_datasets,
)

trainer.train()

model_path = "./nvidia-chatbot-final-model"

trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.475500,0.413159
2,0.315900,0.362258
3,0.207100,0.364819


('./nvidia-chatbot-final-model/tokenizer_config.json',
 './nvidia-chatbot-final-model/special_tokens_map.json',
 './nvidia-chatbot-final-model/spiece.model',
 './nvidia-chatbot-final-model/added_tokens.json',
 './nvidia-chatbot-final-model/tokenizer.json')

In [14]:
eval_results = trainer.evaluate(eval_dataset=test_tokenized_datasets)

In [15]:
eval_results

{'eval_loss': 0.35749316215515137,
 'eval_runtime': 44.5215,
 'eval_samples_per_second': 23.943,
 'eval_steps_per_second': 11.972,
 'epoch': 3.0}

In [20]:
def get_inference(model, tokenizer, prompt):
  tokenized_test_text = tokenizer(prompt,return_tensors='pt').input_ids.to(device)
  model_output = model.generate(tokenized_test_text,
                                      generation_config=GenerationConfig(max_new_tokens=200, num_beams=1),)[0]

  final_output = tokenizer.decode(model_output, skip_special_tokens=True)
  return final_output

In [21]:
question = "what is cuda nsight?"

start_prompt = 'According to the following question:\n\n'
end_prompt = '\n\nAnswer:\n\n'
full_prompt = start_prompt + question + end_prompt
print(full_prompt)

According to the following question:

what is cuda nsight?

Answer:




In [22]:
from transformers import GenerationConfig

finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
base_tokenizer = AutoTokenizer.from_pretrained(model_name)

finetuned_output = get_inference(finetuned_model, tokenizer, full_prompt)
base_output = get_inference(base_model, base_tokenizer, full_prompt)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
base_output

'a snare'

In [23]:
finetuned_output

'cuda nsight is a gpu accelerated nsight tool that provides gpu accelerated profiling and api support for cuda nsight '

In [41]:
test_data[980]

{'question': 'how does azure machine learning enhance the experience of using nvidia ai software ',
 'answer': 'azure machine learning streamlines the deployment of nvidia ai software by integrating it with a cloud based platform for model development ',
 '__index_level_0__': 6502}

In [42]:
question = test_data[980]['question']

start_prompt = 'According to the following question:\n\n'
end_prompt = '\n\nAnswer:\n\n'
full_prompt = start_prompt + question + end_prompt
print(full_prompt)

According to the following question:

how does azure machine learning enhance the experience of using nvidia ai software 

Answer:




In [43]:
finetuned_output = get_inference(finetuned_model, tokenizer, full_prompt)
base_output = get_inference(base_model, base_tokenizer, full_prompt)

In [44]:
base_output

'a new way of interacting with the machine learning model'

In [45]:
finetuned_output

'azure machine learning enhances the experience of using nvidia ai software by providing a cloud based machine learning environment that can seamlessly integrate with azure machine learning '